In [20]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("ainize/klue-bert-base-re")
model = AutoModelForMaskedLM.from_pretrained("ainize/klue-bert-base-re")

print(tokenizer.tokenize("이순신은 [MASK] 중기의 무신이다."))

Some weights of the model checkpoint at ainize/klue-bert-base-re were not used when initializing BertForMaskedLM: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at ainize/klue-bert-base-re and are newly initialized: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
You should probably TRAIN this model on a down-st

['이순신', '##은', '[MASK]', '중기', '##의', '무신', '##이다', '.']


In [ ]:
# import pandas as pd

# df = pd.read_csv("./dataset/test/test_data.csv")
# df.head(5)

In [ ]:
# sentence_list = []
# for i in range(len(df)):
#     sent = df.loc[i, "sentence"]
#     sentence_list.append(sent)

# sentence_list

In [21]:
from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from typing import List

class TaptDataSet(Dataset):
    def __init__(self, file_path_list: List, tokenizer: PreTrainedTokenizer):

        for i in range(len(file_path_list)):
            df = pd.read_csv(file_path_list[i])
            self.sentence_list = []
            for i in range(len(df)):
                sent = df.loc[i, "sentence"]
                self.sentence_list.append(sent)

        self.tokenizer = tokenizer

        self.tokenized_sentences = []

        self.tokenized_sent = self.tokenizer(
            self.sentence_list,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256,
            add_special_tokens=True,
        )

        for idx in range(len(self.tokenized_sent["input_ids"])):
            temp = {key: val[idx] for key, val in self.tokenized_sent.items()}
            self.tokenized_sentences.append(temp)
        

    def __len__(self):
        return len(self.tokenized_sentences)

    def __getitem__(self, idx):
        return self.tokenized_sentences[idx]

In [22]:
from transformers import DataCollatorForLanguageModeling

dataset = TaptDataSet(["./dataset/test/test_data.csv", "./dataset/train/train.csv"], tokenizer=tokenizer)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
# for sent in dataset.tokenized_sentences[:1]:
#     print(sent)

In [ ]:
# print(data_collator(dataset.tokenized_sentences))

In [ ]:
# tokenizer.decode(data_collator(dataset.tokenized_sentences)['input_ids'][0].tolist())

In [23]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='TAPT_roberta',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    learning_rate=5e-6,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [24]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,8.401500
200,7.454300
300,6.987400
400,6.522700
500,6.233200
600,6.083100
700,5.898100
800,5.760400
900,5.639600
1000,5.602100


TrainOutput(global_step=2030, training_loss=5.836645282313154, metrics={'train_runtime': 637.3867, 'train_samples_per_second': 50.942, 'train_steps_per_second': 3.185, 'total_flos': 3855898966051440.0, 'train_loss': 5.836645282313154, 'epoch': 1.0})

In [25]:
trainer.save_model("./TAPT_ainize_output")

In [ ]:
df = pd.read_csv("./dataset/test/test_data.csv")
type(df["sentence"].tolist())